In [8]:
from tqdm import tqdm, trange
import json
from collections import namedtuple, defaultdict
import pandas as pd
import numpy as np
import torch
import os
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.models.word2vec import Word2Vec
import random
from scipy.sparse import csr_matrix
import dgl
import json
from torch_geometric.data import Data

import pickle as pkl
import sys
sys.path.append('/mnt/nfs/zhangtl/utils')
from util import myout

## load

In [9]:
dataset = 'aps'
df = pd.read_csv(f'../../raw_data/{dataset}/Papers.csv')
df.dropna(subset=['p_id', 'date'], inplace=True)
df['year'] = df['date'].apply(lambda u: int(u.split('-')[0]))
df.fillna({'refs': '', 'aids': '', 'v_id': '', 'kids': ''}, inplace=True)
df

/home/ztl/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,title,p_id,date,aids,v_id,kids,refs,year
0,A Study of the Transmission Spectra of Certain...,10.1103/PhysRevSeriesI.1.1,1893-07-01,302635,PRI,,,1893
1,New Books,10.1103/PhysRevSeriesI.1.66,1893-07-01,,PRI,,,1893
2,Note. Geometrical Proof of the Three-Ammeter M...,10.1103/PhysRevSeriesI.1.59,1893-07-01,"302641,302642",PRI,,,1893
3,Notes,10.1103/PhysRevSeriesI.1.62,1893-07-01,,PRI,,,1893
4,On The Infra-red Spectra of the Alkalies,10.1103/PhysRevSeriesI.1.28,1893-07-01,302655,PRI,,,1893
...,...,...,...,...,...,...,...,...
700030,Thermodynamic uncertainty relations for many-b...,10.1103/PhysRevE.104.064141,2021-12-30,"1283804,1283805",PRE,,"10.1103/PhysRevE.49.2726,10.1103/PhysRevLett.8...",2021
700031,Tolman-Ehrenfest-Klein law in non-Riemannian g...,10.1103/PhysRevD.104.124089,2021-12-30,"2008771,2008772",PRD,,"10.1103/PhysRev.35.904,10.1103/PhysRev.36.1791...",2021
700032,Transfer matrix method for light propagation i...,10.1103/PhysRevE.104.064702,2021-12-30,"1285151,1285152,1285153,1285154,1285155,128515...",PRE,,"10.1103/PhysRevLett.25.577,10.1103/PhysRevA.9....",2021
700033,Two-dimensional oxides assembled by <math xmln...,10.1103/PhysRevResearch.3.043231,2021-12-30,"306334,306335,306336,306337,306338",PRRESEARCH,"23,0,5","10.1103/PhysRev.140.A1133,10.1103/PhysRevLett....",2021


In [10]:
start_year, end_year = 2000, 2022
drop_id = df[(df.year < start_year) | (df.year >= end_year)].index
print(f'Drop {len(drop_id)} rows') # 691496
papers = df.drop(drop_id) # 1685097
papers

Drop 293905 rows


,title,p_id,date,aids,v_id,kids,refs,year
293905,<i>Ab initio</i> calculations of phonons in <s...,10.1103/PhysRevB.61.272,2000-01-01,"461571,461572,461573",PRB,,"10.1103/PhysRev.152.705,10.1103/PhysRev.158.43...",2000
293906,<i>Ab initio</i> simulations of compressed liq...,10.1103/PhysRevB.61.909,2000-01-01,"468932,468933,468934,468935",PRB,,"10.1103/PhysRevLett.55.2471,10.1103/PhysRevB.4...",2000
293907,<i>In situ</i> and interrupted-growth studies ...,10.1103/PhysRevE.61.607,2000-01-01,"1266530,1266531,1266532,1266533,1266534",PRE,,"10.1103/PhysRevB.41.1111,10.1103/PhysRevLett.6...",2000
293908,<i>In situ</i> observation of the ferroelectri...,10.1103/PhysRevB.61.203,2000-01-01,"463983,463984,463985,463986,463987",PRB,,"10.1103/PhysRev.144.662,10.1103/PhysRevB.57.16...",2000
293909,"<span class=""aps-inline-formula""><math xmlns=""...",10.1103/PhysRevB.61.1500,2000-01-01,"468371,468372,468373,468374,468375,468376,468377",PRB,,"10.1103/PhysRevB.25.4515,10.1103/PhysRevLett.7...",2000
...,...,...,...,...,...,...,...,...
700030,Thermodynamic uncertainty relations for many-b...,10.1103/PhysRevE.104.064141,2021-12-30,"1283804,1283805",PRE,,"10.1103/PhysRevE.49.2726,10.1103/PhysRevLett.8...",2021
700031,Tolman-Ehrenfest-Klein law in non-Riemannian g...,10.1103/PhysRevD.104.124089,2021-12-30,"2008771,2008772",PRD,,"10.1103/PhysRev.35.904,10.1103/PhysRev.36.1791...",2021
700032,Transfer matrix method for light propagation i...,10.1103/PhysRevE.104.064702,2021-12-30,"1285151,1285152,1285153,1285154,1285155,128515...",PRE,,"10.1103/PhysRevLett.25.577,10.1103/PhysRevA.9....",2021
700033,Two-dimensional oxides assembled by <math xmln...,10.1103/PhysRevResearch.3.043231,2021-12-30,"306334,306335,306336,306337,306338",PRRESEARCH,"23,0,5","10.1103/PhysRev.140.A1133,10.1103/PhysRevLett....",2021


## build graph

In [12]:
dataset = 'aps'
p_id2emb = pkl.load(open(f'../../../01_process/data_relabel/{dataset}/p_id2emb.pkl', 'rb'))
myout(p_id2emb)
def update_idx(idx, dic, cnt, feats, rel, feat_dim, no_emb):
    if idx not in dic:
        dic[idx] = cnt
        cnt += 1
        if rel == 0:
            try:
                feats.append(torch.from_numpy(p_id2emb[idx]).to(torch.float32))
            except:
                feats.append(torch.rand(feat_dim).to(torch.float32)-1)
                no_emb += 1
        elif rel == 1:
            feats.append(torch.rand(feat_dim).to(torch.float32)+1)
        elif rel == 2:
            feats.append(torch.rand(feat_dim).to(torch.float32)+2)
        elif rel == 3:
            feats.append(torch.rand(feat_dim).to(torch.float32))
    return dic, cnt, feats, no_emb

p_id2emb : len=700035, dict([10.1103/PhysRevSeriesI.1.1: [ 6.37484789e-02 -1.56797662e-01 -5.31356275e-01 -2.94902354e-01
  2.31120452e-01  3.06525081e-01 -1.64697647e-01 -1.35186583e-01
  2.44683966e-01  1.04386955e-01  2.70847946e-01 -6.16349354e-02
  3.17574173e-01 -8.29862654e-02  1.76895708e-01 -1.39169414e-02
  1.54908419e-01 -4.55742106e-02 -1.03417464e-01  1.78361490e-01
 -1.57583714e-01  1.14076614e-01 -2.87553906e-01  1.59453943e-01
  1.34424781e-02  1.54614896e-01  8.59246105e-02  8.55579302e-02
 -1.00014448e-01 -8.96584243e-02  4.31428775e-02 -2.55747199e-01
 -3.47675048e-02  2.10791662e-01 -1.00793615e-01  2.39948973e-01
  1.22995190e-01  1.72893703e-01  2.27474242e-01  1.66270465e-01
 -1.22651057e-02 -1.42175406e-01 -1.30287170e-01  1.24966584e-01
  6.04621395e-02  9.48932301e-03  6.88199252e-02  1.25183612e-01
  1.06229082e-01  1.87827960e-01  2.43721791e-02  1.71982035e-01
 -1.64385721e-01  1.04352988e-01  2.46298879e-01  8.15109089e-02
 -1.89954434e-02  8.51822197e-02 

In [14]:
start_year, end_year = 2000, 2022
feat_dim = 128

id2nid, cnt, no_emb = {}, 0, 0
lst, feats = [], []
for i in trange(len(papers)):
    ts = int(papers['year'].iloc[i])
    p_id = str(papers['p_id'].iloc[i])
    refs = papers['refs'].iloc[i]
    
    id2nid, cnt, feats, no_emb = update_idx(p_id, id2nid, cnt, feats, 0, feat_dim, no_emb)
    if len(refs)>0:
        rlst = refs.split(',')
        for ref in rlst:
            if ref != '':
                ref = str(ref)
                id2nid, cnt, feats, no_emb = update_idx(ref, id2nid, cnt, feats, 0, feat_dim, no_emb)
                lst.append((id2nid[p_id], id2nid[ref], 0, ts))
        
feat = torch.stack(feats)
src = torch.tensor([item[0] for item in lst])
tgt = torch.tensor([item[1] for item in lst])
rel = torch.tensor([item[2] for item in lst])
tsp = torch.tensor([item[3] for item in lst])
myout(feat, src, tgt, rel, tsp)

100%|██████████| 406130/406130 [00:26<00:00, 15611.09it/s]


feat : shape=torch.Size([577852, 128])
tensor([[-0.0452, -0.4180, -0.1381,  ..., -0.3730,  0.0763,  0.4313],
        [-0.1554, -0.0979, -0.2336,  ..., -0.1276,  0.1056,  0.3306],
        [-0.0480, -0.4074, -0.1778,  ..., -0.3000,  0.0955,  0.4711],
        ...,
        [-0.0355, -0.1025,  0.0216,  ..., -0.2168,  0.0292,  0.0406],
        [-0.0477, -0.3570, -0.1027,  ..., -0.3685,  0.0045,  0.4755],
        [-0.0142, -0.4941, -0.2021,  ..., -0.3956,  0.0186,  0.4762]])
src : shape=torch.Size([6795779]), tensor([     0,      0,      0,  ..., 577851, 577851, 577851])
tgt : shape=torch.Size([6795779]), tensor([     1,      2,      3,  ..., 540503, 540793, 547191])
rel : shape=torch.Size([6795779]), tensor([0, 0, 0,  ..., 0, 0, 0])
tsp : shape=torch.Size([6795779]), tensor([2000, 2000, 2000,  ..., 2021, 2021, 2021])


In [26]:
graph = dgl.graph((src, tgt))
graph.ndata['feat'] = feat

nid2id = {vv: kk for kk, vv in id2nid.items()}
graph.ndata['raw_nid'] = torch.arange(len(feat))

graph.edata['rel'] = rel
graph.edata['ts'] = tsp
graph

Graph(num_nodes=577852, num_edges=6795779,
      ndata_schemes={'feat': Scheme(shape=(128,), dtype=torch.float32), 'raw_nid': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'rel': Scheme(shape=(), dtype=torch.int64), 'ts': Scheme(shape=(), dtype=torch.int64)})

In [27]:
print(dataset)
dgl.save_graphs(f'../data/{dataset}/graph.bin', [graph])

aps


In [17]:
json.dump(id2nid, open(f'../data/{dataset}/id2nid.json', 'w'))

## gen cites

In [19]:
cites = {}
print(start_year, end_year)
for year in range(start_year, end_year):
    cites[year] = defaultdict(int)

papers.fillna({'refs': ''}, inplace=True)
for i in trange(len(papers)):
    year = int(papers['year'].iloc[i])
    refs = papers['refs'].iloc[i]
    if len(refs)>0:
        rlst = refs.split(',')
        rlst = [str(item) for item in rlst]
        for ref in rlst:
            if ref != '':
                cites[year][ref] += 1

2000 2022


100%|██████████| 406130/406130 [00:11<00:00, 35938.95it/s]


In [20]:
tsp = graph.edata['ts']
ts_vals, ts_cuts = np.unique(tsp.numpy(), return_index=True)
ts_cuts = list(ts_cuts) + [len(tsp.numpy())]

num_ts = len(ts_vals)
ts_infos = np.stack([ts_vals, ts_cuts[0:num_ts], ts_cuts[1:num_ts+1]]).transpose()
myout(ts_cuts, ts_vals, ts_infos)

ts_cuts : len=23, list([0, 172206, 352753, ..., 5821761, 6307991, 6795779])
ts_vals : shape=(22,), [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021]
ts_infos : shape=(22, 3)
[[   2000       0  172206]
 [   2001  172206  352753]
 [   2002  352753  548084]
 [   2003  548084  737985]
 [   2004  737985  952373]
 [   2005  952373 1197814]
 [   2006 1197814 1447421]
 [   2007 1447421 1701523]
 [   2008 1701523 1984965]
 [   2009 1984965 2268813]
 [   2010 2268813 2570824]
 [   2011 2570824 2881569]
 [   2012 2881569 3216577]
 [   2013 3216577 3549432]
 [   2014 3549432 3887154]
 [   2015 3887154 4229233]
 [   2016 4229233 4600367]
 [   2017 4600367 4986306]
 [   2018 4986306 5395358]
 [   2019 5395358 5821761]
 [   2020 5821761 6307991]
 [   2021 6307991 6795779]]


In [21]:
labels = {}
nid2id = {v:k for k,v in id2nid.items()}
for year in range(start_year, end_year):
    left, right = ts_infos[np.where(ts_infos[:, 0]==year)[0][0], 1:]
    nids = graph.edges()[0][left:right].unique().tolist()
    ids = [nid2id[nid] for nid in nids]
    
    pdf = pd.DataFrame({'id': ids, 'nid': nids})
    tbar = trange(year+1, end_year, desc=str(year))
    for yy in tbar:
        cdf = pd.DataFrame({'id': list(cites[yy].keys()), str(yy): list(cites[yy].values())})
        cdf[str(yy)] = cdf[str(yy)].astype('float32')
        
        pdf = pd.merge(pdf, cdf, how='left', on='id')
        tbar.set_postfix(year=year, pdf=len(pdf))
    pdf.fillna(0, inplace=True)
    labels[year] = pdf

2020: 100%|██████████| 1/1 [00:00<00:00,  8.20it/s, pdf=21920, year=2020]
2021: 0it [00:00, ?it/s]


In [22]:
labels[2005]

,id,nid,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,10.1103/PhysRevB.71.014101,201865,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.1103/PhysRevLett.94.016101,201866,1.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.1103/PhysRevB.71.024501,201867,1.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10.1103/PhysRevB.71.035301,201868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,10.1103/PhysRevB.71.014103,201869,2.0,6.0,6.0,3.0,1.0,3.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,5.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17782,10.1103/PhysRevE.72.061112,226938,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17783,10.1103/PhysRevB.72.224439,226939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17784,10.1103/PhysRevC.72.067303,226940,0.0,0.0,1.0,1.0,1.0,0.0,2.0,0.0,2.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0
17785,10.1103/PhysRevC.72.064329,226941,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
pkl.dump(labels, open(f'../data/{dataset}/labels.pkl', 'wb'))

## cum log

In [24]:
def cumulative_log(df):
    colsn = list(df.columns)
    for i in range(3, len(colsn)):
        df[colsn[i]] += df[colsn[i-1]]
    df.iloc[:, 2:] = np.log(df.iloc[:, 2:] + 1)
    return df

labels_cum_log = {}
for year in range(start_year, end_year-2): # 2000, 2020
    labels_cum_log[year] = cumulative_log(labels[year])
labels_cum_log[end_year-2] = labels[end_year-2]
print(len(labels_cum_log))
labels_cum_log[2005]

21


,id,nid,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,10.1103/PhysRevB.71.014101,201865,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
1,10.1103/PhysRevLett.94.016101,201866,0.693147,0.693147,1.098612,1.609438,1.791759,1.791759,1.945910,1.945910,1.945910,2.079442,2.079442,2.079442,2.079442,2.079442,2.079442,2.079442
2,10.1103/PhysRevB.71.024501,201867,0.693147,1.098612,1.098612,1.609438,1.791759,1.945910,1.945910,1.945910,1.945910,1.945910,1.945910,1.945910,1.945910,1.945910,1.945910,1.945910
3,10.1103/PhysRevB.71.035301,201868,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,0.693147,0.693147,0.693147
4,10.1103/PhysRevB.71.014103,201869,1.098612,2.197225,2.708050,2.890372,2.944439,3.091043,3.218876,3.218876,3.258096,3.295837,3.332205,3.367296,3.401197,3.465736,3.610918,3.688879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17782,10.1103/PhysRevE.72.061112,226938,0.000000,0.693147,0.693147,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612
17783,10.1103/PhysRevB.72.224439,226939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
17784,10.1103/PhysRevC.72.067303,226940,0.000000,0.000000,0.693147,1.098612,1.386294,1.386294,1.791759,1.791759,2.079442,2.079442,2.197225,2.197225,2.302585,2.484907,2.484907,2.484907
17785,10.1103/PhysRevC.72.064329,226941,0.693147,0.693147,1.098612,1.098612,1.098612,1.098612,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294


In [25]:
pkl.dump(labels_cum_log, open(f'../data/{dataset}/labels_cum_log.pkl', 'wb'))